In [16]:
import  os
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras
from utils import LoadFishDataUtil
from    tensorflow.keras import datasets, layers, optimizers
from    tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
import pathlib
import h5py
## dataset

## for Model definition/training
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Flatten, Dense, concatenate,  Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint

## required for semi-hard triplet loss:
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
import tensorflow as tf
from tensorflow.keras.models import Sequential
from    tensorflow.keras import datasets, layers, optimizers, models
from    tensorflow.keras import regularizers

## for visualizing 
import matplotlib.pyplot as plt, numpy as np
from sklearn.preprocessing import LabelEncoder
from itertools import islice

le = LabelEncoder()


### include the necessary functions for triplet loss:
** (from TF libraries) **

In [2]:
def pairwise_distance(feature, squared=False):
    """Computes the pairwise distance matrix with numerical stability.

    output[i, j] = || feature[i, :] - feature[j, :] ||_2

    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.

    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
            pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))

    num_data = array_ops.shape(feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances

def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the maximum.

    Returns:
      masked_maximums: N-D `Tensor`.
        The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
        math_ops.multiply(data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the minimum.

    Returns:
      masked_minimums: N-D `Tensor`.
        The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
        math_ops.multiply(data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums

## Define our Triplet Loss

In [3]:
def triplet_loss_adapted_from_tf(y_true, y_pred):
    del y_true
    margin = 1.
    labels = y_pred[:, :1]

 
    labels = tf.cast(labels, dtype='int32')

    embeddings = y_pred[:, 1:]

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

### Define our base_model

In [4]:
def create_base_network(input_shape, embedding_size):
    """
    Base network to be shared (eq. to feature extraction).
    """

    weight_decay = 0.000

    model = Sequential()

    model.add(layers.Conv2D(64, (3, 3), padding='same', input_shape=input_shape,
                            kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    #model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(1024, kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(embedding_size))
    # model.add(layers.Activation('softmax'))
    plot_model(model, to_file='triple_base_network.png', show_shapes=True, show_layer_names=True)
    return model

### Loading the training/validation/testing DATA, as well as some other parameters

In [5]:
#data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_TENT/SESSION1'
data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION_MERGE/SESSION1'
data_dir_path = pathlib.Path(data_dir)
image_count = len(list(data_dir_path.glob('*/*.png')))
print('total images:',image_count)
BATCH_SIZE = 8
IMG_SIZE=160
IMG_WIDTH=320
IMG_HEIGHT=60
IMG_TYPES = ['.png', '.jpg']

total images: 3327


In [6]:
CLASS_NAMES=None
SPLIT_WEIGHTS=(0.7, 0.3, 0.0)# train cv val vs test
myloadData = LoadFishDataUtil(data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES,SPLIT_WEIGHTS)
train_dataset,val_dataset,test_dataset,STEPS_PER_EPOCH, CLASS_NAMES,class_num = myloadData.loadFishDataWithname()
input_shape=(IMG_WIDTH,IMG_HEIGHT, 3)
print('total class:',class_num)


total class: 328


In [7]:
train_dataset_num_elements = tf.data.experimental.cardinality(train_dataset).numpy()
val_dataset_num_elements = tf.data.experimental.cardinality(val_dataset).numpy()
test_dataset_num_elements = tf.data.experimental.cardinality(test_dataset).numpy()
print(f"train_dataset_num_elements {train_dataset_num_elements}, val_dataset_num_elements {val_dataset_num_elements}, test_dataset_num_elements {test_dataset_num_elements}")


train_dataset_num_elements 2328, val_dataset_num_elements 998, test_dataset_num_elements 0


In [47]:
n = 0
x_train = []
y_train = []
inc = iter(train_dataset)
while True:
    try:
        image_batch, label_batch = next(inc)
        imgs = image_batch.numpy()
        labels = label_batch.numpy()
        for i in range(len(imgs)):
            x_train.append(imgs[i,:,:,:])
            y_train.append(labels[i])
            n = n + 1
            #print(n)
    except:
        break
   

print(n)


18624


AttributeError: 'list' object has no attribute 'shape'

In [82]:
n = 0

inc = iter(val_dataset)

while True:
    try:
        image_batch, label_batch = next(inc)
        imgs = image_batch.numpy()
        labels = label_batch.numpy()
        for i in range(len(imgs)):
            if n ==0:
                x_val = np.array(imgs[i])
                y_val = np.array(labels[i])
                x_val = np.expand_dims(x_val,-1)
                y_val = np.expand_dims(y_val,-1)
            else:
                x_val = np.append(x_val, imgs[i], axis = 0)
                y_val = np.append(y_val, labels[i], axis = 0)
            n = n + 1
            print(n)
    except:
        break
   

print(n)


1
1


In [80]:
x_val.shape

(1, 320, 60, 3)

print(x_train.numpy())
print()
print(y_train.numpy())
new_x=[x_train, y_train ]
print(new_x)

In [53]:
batch_size = 8
epochs = 25
train_flag = True  # either     True or False
embedding_size = 128
    
base_network = create_base_network(input_shape, embedding_size)

input_images = Input(shape=input_shape, name='input_image')  # input layer for images
input_labels = Input(shape=(1,), name='input_label')  # input layer for labels
embeddings = base_network([input_images])  # output of network -> embeddings
labels_plus_embeddings = concatenate([input_labels, embeddings])  # concatenating the labels + embeddings

# Defining a model with inputs (images, labels) and outputs (labels_plus_embeddings)
model = Model(inputs=[input_images, input_labels],
              outputs=labels_plus_embeddings)

model.summary()
#plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 320, 60, 3)] 0                                            
__________________________________________________________________________________________________
input_label (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 128)          46325184    input_image[0][0]                
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 129)          0           input_label[0][0]                
                                                                 sequential_1[1][0]         

In [55]:
# train session
opt = Adam(lr=0.0001)  # choose optimiser. RMS is good too!

model.compile(loss=triplet_loss_adapted_from_tf,
              optimizer=opt)

filepath = "model/semiH_trip_fish_ep{epoch:02d}_BS%d.hdf5" % batch_size
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, period=25)
callbacks_list = [checkpoint]

        # Uses 'dummy' embeddings + dummy gt labels. Will be removed as soon as loaded, to free memory
dummy_gt_train = np.zeros((len(x_train), embedding_size + 1))
dummy_gt_val = np.zeros((len(x_val), embedding_size + 1))

x_train = np.reshape(x_train, (len(y_train), x_train.shape[1], x_train.shape[1], 1))
x_val = np.reshape(x_val, (len(y_val), x_train.shape[1], x_train.shape[1], 1))


x_train, y_train = next(iter(train_dataset))
x_val, y_val = next(iter(val_dataset))

# Uses 'dummy' embeddings + dummy gt labels. Will be removed as soon as loaded, to free memory
dummy_gt_train = np.zeros((train_dataset_num_elements, embedding_size + 1))
dummy_gt_val = np.zeros((val_dataset_num_elements, embedding_size + 1))

H = model.fit(
    x=[x_train,y_train],
    y=dummy_gt_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([x_val, y_val], dummy_gt_val),
    callbacks=callbacks_list)

plt.figure(figsize=(8, 8))
plt.plot(H.history['loss'], label='training loss')
plt.plot(H.history['val_loss'], label='validation loss')
plt.legend()
plt.title('Train/validation loss')
plt.show()

ValueError: cannot reshape array of size 460800 into shape (8,320,320,1)

In [ ]:
# build model and optimizer
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()


In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

## training 


In [ ]:
epochs = 20
validation_steps = 20
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# This callback will stop the training when there is no improvement in
# the validation loss for three consecutive epochs.
# train
history=model.fit(train_dataset, epochs=epochs,
                  #callbacks=[callback],
          validation_data=val_dataset, verbose=1, validation_steps=validation_steps)


In [ ]:
# evaluate on test set
#scores = model.evaluate(test_dataset, verbose=1)
#print("Final test loss and accuracy :", scores)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Save model

In [ ]:
# Save the model
# model.save('vggfish.h5py')

In [ ]:
model.save_weights('model/vggfish20191125AQU_MERGE.h5')

In [ ]:
model.load_weights('model/vggfish20191125AQU_MERGE.h5')

# extract feature
## SESSION3

In [ ]:
test_data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION3'
BATCH_SIZE = 8
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES)
sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()
test_num_elements = tf.data.experimental.cardinality(sess2_test_dataset).numpy()
print(f"we have total {test_num_elements} batches of images for testing, around {test_num_elements*BATCH_SIZE} samples")
# evaluate on test set
#scores = model.evaluate(sess2_test_dataset, verbose=1)
#print("Final test loss and accuracy :", scores)
from numpy import linalg as LA
feats = []
names = []
feature_model = models.Model(inputs=model.input, outputs=model.get_layer('batch_normalization_13').output)
n = 0
for image_batch, label_batch in sess2_test_dataset:
    feature=model(image_batch)
    
    #print(n)
    #print(feature.shape[0])
    for i in range(feature.shape[0]):
        n=n+1
        feats.append(feature[i])
        names.append(np.argwhere(label_batch[i]).ravel())
        indxmax=np.argmax(feature[i])
        #print('predictions max index:',indxmax)
        #print('predictions:', CLASS_NAMES[indxmax] )
        #print('real:', CLASS_NAMES[np.argwhere(label_batch[i]).ravel()])
       
print(f"finanly we have {n} samples extracted features")
feats3 = np.array(feats)
names3 = np.array(names)


In [ ]:
test_data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION2'
BATCH_SIZE = 8
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES)
sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()
test_num_elements = tf.data.experimental.cardinality(sess2_test_dataset).numpy()
print(f"we have total {test_num_elements} batches of images for testing, around {test_num_elements*BATCH_SIZE} samples")

feats = []
names = []
feature_model = models.Model(inputs=model.input, outputs=model.get_layer('batch_normalization_13').output)
n = 0
for image_batch, label_batch in sess2_test_dataset:
    feature=model(image_batch)
    for i in range(feature.shape[0]):
        n=n+1
        feats.append(feature[i])
        names.append(np.argwhere(label_batch[i]).ravel())
        indxmax=np.argmax(feature[i])
       
print(f"finanly we have {n} samples extracted features")
feats2 = np.array(feats)
names2 = np.array(names)


In [ ]:
test_data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION1_LT'
BATCH_SIZE = 8
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES)
sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()
test_num_elements = tf.data.experimental.cardinality(sess2_test_dataset).numpy()
print(f"we have total {test_num_elements} batches of images for testing, around {test_num_elements*BATCH_SIZE} samples")

feats = []
names = []
feature_model = models.Model(inputs=model.input, outputs=model.get_layer('batch_normalization_13').output)
n = 0
for image_batch, label_batch in sess2_test_dataset:
    feature=model(image_batch)
    for i in range(feature.shape[0]):
        n=n+1
        feats.append(feature[i])
        names.append(np.argwhere(label_batch[i]).ravel())
        indxmax=np.argmax(feature[i])
       
print(f"finanly we have {n} samples extracted features")
feats1 = np.array(feats)
names1 = np.array(names)





In [ ]:
# asciiList = [n.encode("ascii", "ignore") for n in names]
# directory for storing extracted features
feats_save_path = '/media/xingbo/Storage/fish_identification/data/merge3session.h5'
print ("--------------------------------------------------")
print ("      writing feature extraction results ...")
print ("--------------------------------------------------")

h5f = h5py.File(feats_save_path, 'w')
h5f.create_dataset('Session3_features_merge3session', data=feats3)
h5f.create_dataset('Session3_names_merge3session', data=names3)

h5f.create_dataset('Session2_features_merge3session', data=feats2)
h5f.create_dataset('Session2_names_merge3session', data=names2)

h5f.create_dataset('Session1_features_merge3session', data=feats1)
h5f.create_dataset('Session1_names_merge3session', data=names1)

np.save('/media/xingbo/Storage/fish_identification/data/CLASS_NAMES.npy', CLASS_NAMES)

h5f.close()

# load features

In [ ]:

feats_save_path = '/media/xingbo/Storage/fish_identification/data/merge3session.h5'
print ("--------------------------------------------------")
print ("      reading feature extraction results ...")
print ("--------------------------------------------------")

h5f = h5py.File(feats_save_path, 'r')
Session1_features = h5f['Session1_features_merge3session'][:]
Session1_names = h5f['Session1_names_merge3session'][:]

Session2_features = h5f['Session2_features_merge3session'][:]
Session2_names = h5f['Session2_names_merge3session'][:]

Session3_features = h5f['Session3_features_merge3session'][:]
Session3_names = h5f['Session3_names_merge3session'][:]

h5f.close()
        

# matching session1 vs session 2

In [ ]:
# extract query image's feature, compute simlarity score and sort
scores = np.dot(Session1_features, Session2_features.T)
print(scores.shape)
from numpy.linalg import norm
res = 1 - np.dot(Session1_features/norm(Session1_features, axis=1)[...,None],(Session2_features/norm(Session2_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session1_names == Session2_names.T
lable2 = Session1_names != Session2_names.T

gscores=res[lable]
print(gscores.shape)
iscores=res[lable2]
print(iscores.shape)

In [ ]:
import bob.measure
iscores1vs2 = iscores.astype('float64')
gscores1vs2=gscores.astype('float64') 

In [ ]:
EER = bob.measure.eer(iscores1vs2, gscores1vs2)
print(f"we can achieve EER with {EER}")

In [ ]:
from matplotlib import pyplot
# we assume you have your negatives and positives already split
npoints = 100
bob.measure.plot.roc(iscores1vs2, gscores1vs2, npoints, color=(0,0,0), linestyle='-', label='test') 
pyplot.xlabel('FPR (%)') 
pyplot.ylabel('FNR (%)') 
pyplot.grid(True)
pyplot.show() 

# matching session1 vs session 3


In [ ]:
# extract query image's feature, compute simlarity score and sort
scores = np.dot(Session1_features, Session3_features.T)
print(scores.shape)
from numpy.linalg import norm
res = 1 - np.dot(Session1_features/norm(Session1_features, axis=1)[...,None],(Session3_features/norm(Session3_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session1_names == Session3_names.T
lable2 = Session1_names != Session3_names.T

gscores=res[lable]
print(gscores.shape)
iscores=res[lable2]
print(iscores.shape)

import bob.measure
iscores1vs3 = iscores.astype('float64')
gscores1vs3=gscores.astype('float64') 
EER = bob.measure.eer(iscores1vs3, gscores1vs3)
print(f"we can achieve EER with {EER}")
from matplotlib import pyplot
# we assume you have your negatives and positives already split
npoints = 100
bob.measure.plot.roc(iscores1vs3, gscores1vs3, npoints, color=(0,0,0), linestyle='-', label='test') 
pyplot.xlabel('FPR (%)') 
pyplot.ylabel('FNR (%)') 
pyplot.grid(True)
pyplot.show() 

# matching session2 vs session 3


In [ ]:
# extract query image's feature, compute simlarity score and sort
scores = np.dot(Session2_features, Session3_features.T)
print(scores.shape)
from numpy.linalg import norm
res = 1 - np.dot(Session2_features/norm(Session2_features, axis=1)[...,None],(Session3_features/norm(Session3_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session2_names == Session3_names.T
lable2 = Session2_names != Session3_names.T

gscores=res[lable]
print(gscores.shape)
iscores=res[lable2]
print(iscores.shape)

import bob.measure
iscores2vs3 = iscores.astype('float64')
gscores2vs3=gscores.astype('float64') 
EER = bob.measure.eer(iscores2vs3, gscores2vs3)
print(f"we can achieve EER with {EER}")
from matplotlib import pyplot
# we assume you have your negatives and positives already split
npoints = 100
bob.measure.plot.roc(iscores2vs3, gscores2vs3, npoints, color=(0,0,0), linestyle='-', label='test') 
pyplot.xlabel('FPR (%)') 
pyplot.ylabel('FNR (%)') 
pyplot.grid(True)
pyplot.show() 

In [ ]:
from matplotlib import pyplot
# we assume you have your negatives and positives already split
npoints = 100
bob.measure.plot.roc(iscores1vs2, gscores1vs2, npoints, color=(0,0,0), linestyle='-', label='1 vs 2') 
bob.measure.plot.roc(iscores1vs3, gscores1vs3, npoints, color=(0,0,0), linestyle='-.', label='1 vs 3') 
bob.measure.plot.roc(iscores2vs3, gscores2vs3, npoints, color=(0,0,0), linestyle=':', label='2 vs 3') 
pyplot.xlabel('FPR (%)') 
pyplot.ylabel('FNR (%)') 
pyplot.legend()
pyplot.grid(True)
pyplot.show() 


# intra session

In [ ]:
import bob.measure
from matplotlib import pyplot
from numpy.linalg import norm



res = 1 - np.dot(Session1_features/norm(Session1_features, axis=1)[...,None],(Session1_features/norm(Session1_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session1_names == Session1_names.T
lable2 = Session1_names != Session1_names.T

gscores=res[lable]
iscores=res[lable2]

iscores1vs1 = iscores.astype('float64')
gscores1vs1=gscores.astype('float64') 
EER1vs1 = bob.measure.eer(iscores1vs1, gscores1vs1)
print(f"we can achieve EER with {EER1vs1}")

res = 1 - np.dot(Session2_features/norm(Session2_features, axis=1)[...,None],(Session2_features/norm(Session2_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session2_names == Session2_names.T
lable2 = Session2_names != Session2_names.T

gscores=res[lable]
iscores=res[lable2]

iscores2vs2 = iscores.astype('float64')
gscores2vs2=gscores.astype('float64') 
EER2vs2 = bob.measure.eer(iscores2vs2, gscores2vs2)
print(f"we can achieve EER with {EER2vs2}")

res = 1 - np.dot(Session3_features/norm(Session3_features, axis=1)[...,None],(Session3_features/norm(Session3_features,axis=1)[...,None]).T)
res = 1-res/2
lable = Session3_names == Session3_names.T
lable2 = Session3_names != Session3_names.T

gscores=res[lable]
iscores=res[lable2]

iscores3vs3 = iscores.astype('float64')
gscores3vs3=gscores.astype('float64') 
EER3vs3 = bob.measure.eer(iscores3vs3, gscores3vs3)
print(f"we can achieve EER with {EER3vs3}")


from matplotlib import pyplot
# we assume you have your negatives and positives already split
npoints = 100
bob.measure.plot.roc(iscores1vs1, gscores1vs1, npoints, color=(0,0,0), linestyle='-', label='inta-session1') 
bob.measure.plot.roc(iscores2vs2, gscores2vs2, npoints, color=(0,0,0), linestyle='-.', label='inta-session2') 
bob.measure.plot.roc(iscores3vs3, gscores3vs3, npoints, color=(0,0,0), linestyle=':', label='inta-session3') 
pyplot.xlabel('FPR (%)') 
pyplot.ylabel('FNR (%)') 
pyplot.legend()
pyplot.grid(True)
pyplot.show() 